In [1]:
import os
import glob
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
if not os.path.exists('./../output/only_pieces/'):
    os.mkdir('./../output/only_pieces/')


In [12]:
'''   fen_to_piece = {
            'K': 'w_King', 'Q': 'w_Queen', 'R': 'w_Rook', 'B': 'w_Bishop', 'N': 'w_Knight', 'P': 'w_Pawn',
            'k': 'b_King', 'q': 'b_Queen', 'r': 'b_Rook', 'b': 'b_Bishop', 'n': 'b_Knight', 'p': 'b_Pawn'
        }
'''
if not os.path.exists(f'./../output/only_pieces'):
        os.mkdir(f'./../output/only_pieces')

classes = ["w_Pawn", "w_Knight", "w_Bishop", "w_Rook", "w_Queen", "w_King",
                   "b_Pawn", "b_Knight", "b_Bishop", "b_Rook", "b_Queen", "b_King"]
for c in classes:
    if not os.path.exists(f'./../output/only_pieces/{c}'):
        os.mkdir(f'./../output/only_pieces/{c}')


In [13]:
to_be_processed = glob.glob('./../output/training_pieces/**')
coor_content_matrix = []
for in_process in to_be_processed[1::2]:
    if not os.path.isfile(in_process):
        continue
        
    if in_process.lower().endswith(".png"):
        continue

    #find the base name and open related txt file
    base_name = os.path.basename(in_process)
    name_parts = os.path.splitext(base_name)[0].split('.txt')

    # Open the corresponding text file
    if os.path.isfile(in_process):
        with open(in_process, 'r') as txt_file:
            content = txt_file.read()
    else:
        print(f"Text file not found for {base_name}")

    path_file_png_origine = os.path.join('./../output/training_pieces/', f'{name_parts[0]}.png')
    path_file_png_destinazione = os.path.join(f'./../output/only_pieces/{content}/', f'{name_parts[0]}.png')
    
    shutil.copy(path_file_png_origine, path_file_png_destinazione)


[]


In [2]:

classes = ["w_Pawn", "w_Knight", "w_Bishop", "w_Rook", "w_Queen", "w_King",
                   "b_Pawn", "b_Knight", "b_Bishop", "b_Rook", "b_Queen", "b_King"]
for c in classes:
    
    display("Numero di sample della classe "+ str(c)+ ": "+ str(len(os.listdir(f'./../output/only_pieces/{c}'))))


'Numero di sample della classe w_Pawn: 68'

'Numero di sample della classe w_Knight: 14'

'Numero di sample della classe w_Bishop: 14'

'Numero di sample della classe w_Rook: 19'

'Numero di sample della classe w_Queen: 10'

...

'Numero di sample della classe b_Knight: 12'

'Numero di sample della classe b_Bishop: 15'

'Numero di sample della classe b_Rook: 19'

'Numero di sample della classe b_Queen: 10'

'Numero di sample della classe b_King: 12'

In [3]:
dataset_path = './../output/only_pieces/'
class_names = ["w_Pawn", "w_Knight", "w_Bishop", "w_Rook", "w_Queen", "w_King",
                   "b_Pawn", "b_Knight", "b_Bishop", "b_Rook", "b_Queen", "b_King"]
data = []
labels = []
for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        data.append(img_path)
        labels.append(class_name)

data, labels = shuffle(data, labels, random_state=42)

# Split the data into training and temporary sets
train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=0.3, stratify=labels, random_state=42)

# Split the temporary set into validation and test sets
test_data, validation_data, test_labels, validation_labels = train_test_split(temp_data, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)

display("Lunghezza train data: "+ str(len(train_data)))
display("Lunghezza validation data: " +str(len(validation_data)))
display("Lunghezza test data: " + str(len(test_data)))
#display(test_data)

'Lunghezza train data: 192'

'Lunghezza validation data: 42'

'Lunghezza test data: 41'

In [4]:
class ChessPicesDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]

        # Carica l'immagine da file
        img = Image.open(img_path)

        # Applica le trasformazioni se specificate
        if self.transform:
            img = self.transform(img)

        return img, label

In [5]:
# Definisci le trasformazioni per il tuo dataset (es. resizing, normalizzazione, ecc.)
transform = transforms.Compose([
    transforms.Resize((100,200)),
    transforms.ToTensor()
])

In [6]:
train_ds = ChessPicesDataset(train_data, train_labels, transform=transform)
validation_ds = ChessPicesDataset(validation_data, validation_labels, transform=transform)
test_ds = ChessPicesDataset(test_data, test_labels, transform=transform)

In [7]:
# Crea i dataloader
batch_size = 32
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_ds, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [8]:
# Load the pre-trained ResNet18 model
resnet_model = models.resnet18(pretrained=True)

# Modify the final fully connected layer for the number of classes in your dataset
num_classes = len(set(train_ds.labels))

#print(len(img), len(set(lable)))
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

# Move the model to the specified device
resnet_model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

c:\Users\feder\anaconda3\envs\LabCv2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\feder\anaconda3\envs\LabCv2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Fine-tune the model
num_epochs = 10

# Liste per salvare i valori di loss
train_losses = []
validation_losses = []

for epoch in range(num_epochs):
    resnet_model.train()
    for inputs, labels in train_dataloader:
        #inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, torch.tensor([class_names.index(label) for label in labels]))
        loss.backward()
        optimizer.step()
    # Validate the model
    resnet_model.eval()
    with torch.no_grad():
        valid_loss = 0.0
        for inputs, labels in validation_dataloader:
            outputs = resnet_model(inputs)
            valid_loss += criterion(outputs, torch.tensor([class_names.index(label) for label in labels]))

    # Calcola la media della loss sul set di validazione
    valid_loss /= len(validation_dataloader)

    # Aggiorna le liste di loss
    train_losses.append(loss.item())
    validation_losses.append(valid_loss.item())

    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {valid_loss.item()}')
    

resnet_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = resnet_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += len(labels)
        correct += (predicted == torch.tensor([class_names.index(label) for label in labels])).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Save the fine-tuned model if needed
torch.save(resnet_model.state_dict(), 'fine_tuned_resnet18.pth')



Epoch 1/10, Training Loss: 1.4326143264770508, Validation Loss: 6.417305946350098
Epoch 2/10, Training Loss: 1.611130952835083, Validation Loss: 7.438599586486816
Epoch 3/10, Training Loss: 0.35469529032707214, Validation Loss: 1.2338249683380127
Epoch 4/10, Training Loss: 0.19556979835033417, Validation Loss: 1.653531551361084
Epoch 5/10, Training Loss: 0.1379767209291458, Validation Loss: 1.310896873474121
Epoch 6/10, Training Loss: 0.00866552721709013, Validation Loss: 1.390994668006897
Epoch 7/10, Training Loss: 0.05487512797117233, Validation Loss: 1.1635704040527344
Epoch 8/10, Training Loss: 0.021055564284324646, Validation Loss: 2.0351662635803223
Epoch 9/10, Training Loss: 0.006859513930976391, Validation Loss: 1.1216256618499756
Epoch 10/10, Training Loss: 0.061165887862443924, Validation Loss: 1.315464973449707
Test Accuracy: 70.73%
